# Classification with LSTM and GBM on Selected Features
This notebook loads features selected by the Genetic Algorithm and evaluates classification performance using LSTM and GBM models.

In [ ]:
import numpy as np
import pickle
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input


In [ ]:
# Load compressed features and selected feature mask
with open("cbam_compressed_features.pkl", "rb") as f:
    data = pickle.load(f)
    X = data['compressed_features']
    y = data['y_train']

with open("selected_features_ga.pkl", "rb") as f:
    mask = pickle.load(f)['mask']

X_selected = X[:, mask == 1]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

## Gradient Boosting Classifier (GBM)

In [ ]:
gbm = GradientBoostingClassifier()
gbm.fit(X_train, y_train)
y_pred_gbm = gbm.predict(X_test)
print("GBM Accuracy:", accuracy_score(y_test, y_pred_gbm))
print("GBM F1 Score:", f1_score(y_test, y_pred_gbm))

## Long Short-Term Memory (LSTM) Network

In [ ]:
# Reshape for LSTM [samples, timesteps, features] where timesteps = 1
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define LSTM model
model = Sequential([
    Input(shape=(1, X_train.shape[1])),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_lstm, y_train, epochs=10, batch_size=16, validation_split=0.1, verbose=1)

y_pred_lstm = (model.predict(X_test_lstm) > 0.5).astype(int).flatten()
print("LSTM Accuracy:", accuracy_score(y_test, y_pred_lstm))
print("LSTM F1 Score:", f1_score(y_test, y_pred_lstm))